<a href="https://colab.research.google.com/github/Deepanshu0810/pdf-chatbot/blob/main/pdf_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -r /content/drive/MyDrive/pdf-gpt/requirement.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.3 MB/s eta 0:00:00


In [2]:
import torch
import PyPDF2 # pdf reader
import time
from pypdf import PdfReader
from io import BytesIO
from langchain.prompts import PromptTemplate # for custom prompt specification
from langchain.text_splitter import RecursiveCharacterTextSplitter # splitter for chunks
from langchain.embeddings import HuggingFaceEmbeddings # embeddings
from langchain.vectorstores import FAISS # vector store database
from langchain.chains import RetrievalQA # qa and retriever chain
from langchain.memory import ConversationBufferMemory # for model's memory on past conversations
from langchain.document_loaders import PyPDFDirectoryLoader # loader fo files from firectory

from langchain.llms.huggingface_pipeline import HuggingFacePipeline # pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [3]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "Deci/DeciLM-7B-instruct" # model repo id
device = 'cuda'

#
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             trust_remote_code=True,
                                             device_map = "auto",
                                          quantization_config=quant_config)

# create a pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                temperature=1e-3,
                return_full_text = False,
                max_new_tokens=2048)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
from langchain.document_loaders import ArxivLoader
from langchain.document_loaders.merge import MergedDataLoader
doc_to_query = ["2312.00761", "2302.09880", "2302.12448", "2106.15093", "1703.00810"]
docs_to_merge = []
for doc in doc_to_query:
    loader = ArxivLoader(query=doc)
    docs_to_merge.append(loader)
all_loaders = MergedDataLoader(loaders=docs_to_merge)
all_docs = all_loaders.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=16)
splits = text_splitter.split_documents(all_docs)
print(f"We have {len(splits)} chunks in memory")

We have 397 chunks in memory


In [9]:
embeddings_model_id = "BAAI/bge-large-en-v1.5"

embeddings_model = HuggingFaceEmbeddings(model_name=embeddings_model_id,model_kwargs = {"device": "cuda"})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
vectorstore_db = FAISS.from_documents(splits, embeddings_model)

In [25]:
vectorstore_db.save_local("/content/drive/MyDrive/pdf-gpt/vectorstore_db.index")

In [27]:
# performs a similarity check and returns the top K embeddings
# that are similar to the question's embeddings
retriever = vectorstore_db.as_retriever(search_type="mmr",
                                        search_kwargs={"k": 8})

In [28]:
retrieved_relevant_docs = retriever.get_relevant_documents(
    "What are the core concepts of unlearning in deep learning as presented in these papers, and how do the approaches differ or align?"
)

for doc in retrieved_relevant_docs:
    print(doc.page_content)
    print('\n')

Given this pipeline, we evaluate three methods, namely Fisher,
Influence, and DeltaGrad, that follow largely different ap-
proaches for machine unlearning and represent the state of the
art for our setting (linear classification models trained with SGD).
Fisher updates the initial ML model using the remaining data to
perform a corrective Newton step; it follows Golatkar et al. [11].
Influence updates the initial ML model using the deleted data to
perform a corrective Newton step; it is defined in Guo et al. [14].
And DeltaGrad updates the initial ML model by correcting the
SGD steps that led to the initial model; it follows the method defined
in Wu et al. [32]. Note that, in this work, we extend the original
papers of Wu et al. [32] and Golatkar et al. [11], so as to ensure that
all the evaluated methods are equipped with mechanisms to control
the trade-offs between efficiency, effectiveness and certifiability.
For the experimental evaluation, we implement the three meth-


the IEEE/CV

In [29]:
custom_prompt_template = """
### System:
You are an AI assistant that follows instructions extremely well. Help as much as you can.
### User:
You are a research assistant for an artificial intelligence student. Use only the following information to answer user queries:
Context= {context}
History = {history}
Question= {question}
### Assistant:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=["question", "context", "history"])

In [30]:
memory = ConversationBufferMemory(input_key="question",
                                   memory_key="history",
                                   return_messages=True)

In [32]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstore_db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={
        "verbose": False,
        "prompt": prompt,
        "memory": memory
    }
)


In [33]:
query = "What are the core concepts of unlearning in deep learning as presented in these papers, and how do the approaches differ or align?"
response = qa_chain({"query": query})
response['result']

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' The core concepts of unlearning in deep learning, as presented in these papers, are the methods and approaches used to remove the effects of deleted data from a machine learning model. The approaches differ in their specific techniques for removing the effects of deleted data, such as Fisher updates, Influence updates, and DeltaGrad updates. These methods aim to maintain the effectiveness of the model while minimizing the impact of deleting data on its performance. The approaches also differ in their control of trade-offs between efficiency, effectiveness, and certifiability.'